### Gold Layer Analytics – TelecomSparkTransformations

This notebook creates analytics-ready Gold Delta tables
from Silver data using Spark SQL and Window functions.

Principles:
- Business-level metrics
- Aggregations and window functions
- Optimized for reporting & BI


**Inspect Silver Data**

In [0]:
spark.table("telecomsparktransformations_catalog.silver.usage_enriched").show()

**Subscriber-Level Metrics (Aggregation)**
  Create per-subscriber usage metrics.

In [0]:
%sql
CREATE OR REPLACE TABLE telecomsparktransformations_catalog.gold.subscriber_metrics AS
SELECT
    subscriber_id,
    name,
    plan_type,
    circle,
    COUNT(DISTINCT usage_date) AS active_days,
    SUM(total_usage_mb)        AS total_usage_mb,
    AVG(total_usage_mb)        AS avg_daily_usage_mb
FROM telecomsparktransformations_catalog.silver.usage_enriched
GROUP BY
    subscriber_id,
    name,
    plan_type,
    circle;


**Exam Rule**
Reporting aggregations → Spark SQL preferred

 **Validate Subscriber Metrics**

In [0]:
%sql
SELECT * FROM telecomsparktransformations_catalog.gold.subscriber_metrics;

##### Daily Network Usage (Time-Based Aggregation)

  - Total telecom network usage per day.


In [0]:
%sql
CREATE OR REPLACE TABLE telecomsparktransformations_catalog.gold.daily_network_usage AS
SELECT
    usage_date,
    SUM(total_usage_mb) AS total_network_usage_mb,
    COUNT(DISTINCT subscriber_id) AS active_subscribers
FROM telecomsparktransformations_catalog.silver.usage_enriched
GROUP BY usage_date
ORDER BY usage_date;


**Validate Daily Network Usage**

In [0]:
%sql
SELECT * FROM telecomsparktransformations_catalog.gold.daily_network_usage;

**Subscriber Ranking (Window Function)**
-   Rank subscribers by total usage.

In [0]:
%sql
SELECT
    subscriber_id,
    name,
    total_usage_mb,
    RANK() OVER (ORDER BY total_usage_mb DESC) AS usage_rank
FROM telecomsparktransformations_catalog.gold.subscriber_metrics;


**Exam Favorite**
- RANK() → gaps
- DENSE_RANK() → no gaps
- ROW_NUMBER() → unique ranking

**Rolling 7-Day Usage (Window Function)**
  - Calculate rolling usage trend per subscriber.

In [0]:
%sql
SELECT
    subscriber_id,
    usage_date,
    total_usage_mb,
    SUM(total_usage_mb) OVER (
        PARTITION BY subscriber_id
        ORDER BY usage_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS rolling_7_day_usage_mb
FROM telecomsparktransformations_catalog.silver.usage_enriched
ORDER BY subscriber_id, usage_date;


**Exam Rule**
  - Rolling metrics ALWAYS require ORDER BY in window